In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20140101
cursor = dbop.connect_db("sqlite3").cursor()

t0 = time.time()
df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
print('读取：',time.time()-t0)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
print('预处理：',time.time()-t0)
df_d = df_d.drop(columns=['open0','high0','low0','vol0'])
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5

df_d = df_d.sort_index()
df_d['prev_close'] = df_d['close'].sort_index().groupby('code').shift(1)
print('数据准备：',time.time()-t0)
df_d[['close','prev_close']].head()


df_d['day_pct'] = ((df_d['close']-df_d['open'])/df_d['prev_close'])*100
df_d['prev_pct'] = df_d['pct'].sort_index().groupby('code').shift(1)
df_d['prev_day_pct'] = df_d['day_pct'].sort_index().groupby('code').shift(1)
df_d[['prev_close','open','close','pct','prev_pct','day_pct','prev_day_pct']].head()
print('日内涨幅：',time.time()-t0)

##
periods = [5,10,20,30,40,60,120,250]
for k in periods:
#     df_d['{:d}ma'.format(k)]=df_d.reset_index('code').groupby('code')['close'].rolling(k).mean()
    df_d_tmp = groupby_rolling(df_d[['close']],level="code",window=k,ops='mean',min_periods=min_periods)
    df_d['{:d}ma'.format(k)] = df_d_tmp[df_d_tmp.columns[0]]
#     df_d['{:d}ma_avg'.format(k)]=df_d.reset_index('code').groupby('code')['amt'].rolling(k).sum()\
#             /df_d.reset_index('code').groupby('code')['vol'].rolling(k).sum()*1e6
print(df_d.columns)
print("均线数据准备："，time.time()-t0)

for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df_d['{}/{}'.format(col1,col2)] = df_d[col1]/df_d[col2]
# for col2 in ["{}ma_avg".format(k) for k in periods]:
#     col1 = 'close'
#     df_d['{}/{}'.format(col1,col2)] = df_d[col1]/df_d[col2]
print("均线相对距离计算："，time.time()-t0)
print(df_d.columns)

df_d_d[['30ma_pct','60ma_pct']] = df_d_d[['30ma','60ma']].sort_index().groupby('code').pct_change()*100 # 13s
print("均线涨幅计算："，time.time()-t0)


df_d = df_d.astype('float32')
print("数据压缩："，time.time()-t0)
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')
df_d.tail()

select * from stock_day where date>=20140101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(4896413, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4896413 entries, (000001.SZ, 2014-01-02 00:00:00) to (688598.SH, 2020-05-29 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 384.6 MB


open       high        low      close  \
code      date                                                     
688598.SH 2020-05-25  88.669998  88.669998  76.720001  78.300003   
          2020-05-26  78.470001  81.500000  77.510002  79.580002   
          2020-05-27  82.040001  93.150002  82.040001  90.300003   
          2020-05-28  93.059998  94.970001  86.379997  92.360001   
          2020-05-29  88.000000  91.330002  87.000000  88.099998   

                               vol       amt  adj_factor        avg  \
code      date                                                        
688598.SH 2020-05-25  53744.101562  4.323187         1.0  80.440216   
          2020-05-26  35312.359375  2.802792         1.0  79.371414   
          2020-05-27  75112.562500  6.597441         1.0  87.834061   
          2020-05-28  69327.757812  6.323590         1.0  91.212959   
          2020-05-29  45330.218750  4.034555         1.0  89.003647   

                         close0       avg0        pct  
code      date                                         
688598.SH 2020-05-25  78.300003  80.440216 -11.022722  
          2020-05-26  79.580002  79.371414   1.634741  
          2020-05-27  90.300003  87.834061  13.470721  
          2020-05-28  92.360001  91.212959   2.281284  
          2020-05-29  88.099998  89.003647  -4.612386

In [4]:
df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')
df_d_basic.tail()

select * from stock_daily_basic where date>=20140101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(4594496, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4594496 entries, (000001.SZ, 2014-01-02 00:00:00) to (688598.SH, 2020-05-29 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 298.2 MB


close  turnover_rate  turnover_rate_f  volume_ratio  \
code      date                                                                  
688598.SH 2020-05-25  78.300003      29.393801        29.393801          0.67   
          2020-05-26  79.580002      19.313101        19.313101          0.55   
          2020-05-27  90.300003      41.080700        41.080700          1.43   
          2020-05-28  92.360001      37.916901        37.916901          1.34   
          2020-05-29  88.099998      24.792101        24.792101          0.82   

                             pe     pe_ttm      pb         ps     ps_ttm  \
code      date                                                             
688598.SH 2020-05-25  80.646301  67.683403  5.3239  26.152000  23.463699   
          2020-05-26  81.964699  68.789803  5.4109  26.579500  23.847200   
          2020-05-27  93.005898  78.056297  6.1398  30.159901  27.059601   
          2020-05-28  95.127701  79.836998  6.2799  30.848000  27.677000   
          2020-05-29  90.739998  76.154602  5.9902  29.425100  26.400400   

                      total_share  float_share   free_share  total_mv  \
code      date                                                          
688598.SH 2020-05-25       8000.0  1828.415039  1828.415039  626400.0   
          2020-05-26       8000.0  1828.415039  1828.415039  636640.0   
          2020-05-27       8000.0  1828.415039  1828.415039  722400.0   
          2020-05-28       8000.0  1828.415039  1828.415039  738880.0   
          2020-05-29       8000.0  1828.415039  1828.415039  704800.0   

                            circ_mv   pb*pe_ttm       pb*pe  
code      date                                               
688598.SH 2020-05-25  143164.890625  360.339691  429.352875  
          2020-05-26  145505.265625  372.214752  443.502808  
          2020-05-27  165105.875000  479.250061  571.037598  
          2020-05-28  168872.406250  501.368378  597.392456  
          2020-05-29  161083.359375  456.181305  543.550720

In [5]:
t0 = time.time()
df = df_d.join(df_d_basic.drop(columns=['close']),how='left')
print(time.time()-t0)
df.shape

d_basic_cols = df_d_basic.columns
del df_d_basic

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '10ma', '20ma', '40ma', '60ma', '120ma', '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '10ma', '20ma', '40ma', '60ma', '120ma', '250ma',
       'close/5ma', 'close/10ma', 'close/20ma', 'close/40ma', 'close/60ma',
       'close/120ma', 'close/250ma'],
      dtype='object')


In [286]:
start = 20140101
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
t0 = time.time()
df_r = pd.read_parquet(r"database\return_spc_5%_10%_20_8_inf%").loc[idx[:,start_date:],:].sort_index()
# df_r = pd.read_parquet(r"database\return_spc_5%_30%_60_8_inf%").loc[idx[:,start_date:],:].sort_index()
print(time.time()-t0)
df_r.shape
df_r.head()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


23.825343370437622


(4593560, 11)

sell_at  sell_at0  sell_date  sell_idx      buy_at  \
code      date                                                                
000001.SZ 2014-01-02  680.792419     11.66 2014-01-16     908.0  709.402039   
          2014-01-03  678.456970     11.62 2014-01-17     909.0  694.221436   
          2014-01-06  663.276306     11.36 2014-01-22     912.0  673.202087   
          2014-01-07  660.940857     11.32 2014-01-21     911.0  679.624695   
          2014-01-08  660.940857     11.32 2014-01-21     911.0  682.544006   

                      buy_at0         max  max_idx  idx  is_selled         r  
code      date                                                                
000001.SZ 2014-01-02    12.15  709.985901    899.0  898       True -0.040329  
          2014-01-03    11.89  700.644043    900.0  899       True -0.022708  
          2014-01-06    11.53  700.060120    903.0  900       True -0.014744  
          2014-01-07    11.64  700.060120    903.0  901       True -0.027491  
          2014-01-08    11.69  700.060120    903.0  902       True -0.031651

In [287]:
t0 = time.time()
df = df.join(df_r,how='inner')
print(time.time()-t0)
df.shape

## 更新df_r
# t0 = time.time()
# df = df.drop(columns=r_columns).join(df_r,how='inner')
# print(time.time()-t0)
# df.shape

r_columns = df_r.columns
del df_r

21.469016790390015


(4593560, 64)

In [8]:
cols = df.columns[df.dtypes=='float64']
cols
t0 = time.time()
df[cols] = df[cols].astype('float32') # 3.8s
print(time.time()-t0)


In [296]:
# df.info()
# df.columns
# df.dtypes
t0 = time.time()
df.to_hdf(os.path.join('datasets','df_20140101.h5'),key='data')
print(time.time()-t0)

# t0 = time.time()
# df = pd.read_hdf(os.path.join('datasets','df_20140101.h5'),key='data')
# print(time.time()-t0)

# df.shape
# df.head()
# df.columns


34.069068908691406


In [15]:
r_columns = ['buy_at','max','max_idx', 'idx', 'is_selled', 'sell_at', 'sell_date', 'r']

In [13]:
t0 = time.time()
df_pct_amt = df[['pct','day_pct','amt']].sort_index().groupby('code').shift(1).abs()
print(time.time()-t0)
df_pct_amt = df_pct_amt.add_prefix('abs_')
df_pct_amt.head()

min_periods=1

df_pct_amt5 = groupby_rolling(df_pct_amt,level="code",window=5,ops=['max','mean'],min_periods=min_periods)
df_pct_amt10 = groupby_rolling(df_pct_amt,level="code",window=10,ops=['max','mean'],min_periods=min_periods)
# 19s
print(time.time()-t0) 
df_pct_amt5.head()

5.703505516052246


abs_pct  abs_day_pct   abs_amt
code      date                                       
000001.SZ 2014-01-02       NaN          NaN       NaN
          2014-01-03       NaN          NaN  5.962237
          2014-01-06  2.452976     1.798849  6.566313
          2014-01-07  2.179384     1.844092  6.792804
          2014-01-08  0.342757     0.856905  3.939776

26.24134922027588


p5max_abs_day_pct  p5mean_abs_day_pct  p5max_abs_amt  \
code      date                                                               
000001.SZ 2014-01-02                NaN                 NaN            NaN   
          2014-01-03                NaN                 NaN            NaN   
          2014-01-06                NaN                 NaN            NaN   
          2014-01-07                NaN                 NaN            NaN   
          2014-01-08           1.844092            1.499949       6.792804   

                      p5mean_abs_amt  p5max_abs_pct  p5mean_abs_pct  
code      date                                                       
000001.SZ 2014-01-02             NaN            NaN             NaN  
          2014-01-03             NaN            NaN             NaN  
          2014-01-06             NaN            NaN             NaN  
          2014-01-07             NaN            NaN             NaN  
          2014-01-08        5.815283       2.452976        1.658372

In [9]:
mask_date = df.index.get_level_values('date')>='2016-01-01'
df_baseline = df.loc[mask_date,'r'].agg(agg_operations[:-1])
df_baseline_q = df.loc[mask_date,'r'].reset_index('code').resample('Q').agg(agg_operations)
df_baseline
df_baseline_q

mean     -0.002133
median   -0.016800
max       5.111110
min      -0.972953
std       0.081040
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2016-03-31  0.001179 -0.013889  2.548668 -0.972953  0.094469  150433
2016-06-30  0.000853 -0.015754  1.533937 -0.714186  0.080108  155157
2016-09-30  0.005089 -0.006729  1.639485 -0.651399  0.066105  170115
2016-12-31 -0.004053 -0.014433  3.136834 -0.900237  0.068697  165772
2017-03-31  0.001947 -0.009050  1.347707 -0.482470  0.070502  170618
2017-06-30 -0.013016 -0.025000  5.111110 -0.852388  0.072342  178502
2017-09-30  0.005735 -0.006309  2.099174 -0.804020  0.070351  200686
2017-12-31 -0.016474 -0.027645  2.659847 -0.814664  0.063496  193463
2018-03-31 -0.006932 -0.022059  2.229388 -0.967228  0.086282  190955
2018-06-30 -0.022454 -0.030781  1.119939 -0.950667  0.067774  197214
2018-09-30 -0.020828 -0.027933  1.655462 -0.869820  0.058617  218246
2018-12-31  0.003390 -0.016175  2.749064 -0.612903  0.083702  210707
2019-03-31  0.044422  0.014218  3.320968 -0.746753  0.117071  206870
2019-06-30 -0.011983 -0.022901  1.897051 -0.861940  0.067602  215321
2019-09-30  0.000302 -0.017704  2.243787 -0.876993  0.075026  236958
2019-12-31  0.010891 -0.003697  2.125000 -0.731959  0.078266  225939
2020-03-31 -0.012337 -0.023256  1.891414 -0.763889  0.110282  218582
2020-06-30 -0.005392 -0.020305  1.417069 -0.787500  0.072459  148256

In [28]:
mask_date2 = df.index.get_level_values('date')>='2018-01-01'
df_baseline2 = df.loc[mask_date2,'r'].agg(agg_operations[:-1])
df_baseline_q2 = df.loc[mask_date2,'r'].reset_index('code').resample('Q').agg(agg_operations)
df_baseline2
df_baseline_q2

mean     -0.001853
median   -0.018078
max       3.320968
min      -0.967228
std       0.085897
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2018-03-31 -0.006932 -0.022059  2.229388 -0.967228  0.086282  190955
2018-06-30 -0.022454 -0.030781  1.119939 -0.950667  0.067774  197214
2018-09-30 -0.020828 -0.027933  1.655462 -0.869820  0.058617  218246
2018-12-31  0.003390 -0.016175  2.749064 -0.612903  0.083702  210707
2019-03-31  0.044422  0.014218  3.320968 -0.746753  0.117071  206870
2019-06-30 -0.011983 -0.022901  1.897051 -0.861940  0.067602  215321
2019-09-30  0.000302 -0.017704  2.243787 -0.876993  0.075026  236958
2019-12-31  0.010891 -0.003697  2.125000 -0.731959  0.078266  225939
2020-03-31 -0.012337 -0.023256  1.891414 -0.763889  0.110282  218582
2020-06-30 -0.005392 -0.020305  1.417069 -0.787500  0.072459  148256

In [45]:
len(mask_date2)
((df['close/60ma']<1.05) & (df['close/30ma']<1.05) &mask_date2).sum()

4593560

1392888

In [105]:
# mean     -0.001853
# median   -0.018078
# size     4593560


# mean         -0.003998
# median       -0.021001
# max           1.964467
# min          -0.675676
# std           0.089355
# size      40895.000000
# Name: r, dtype: float64
mask_big_rise = [
    # -0.003998 40895
    # 实体大阳线
    df['day_pct']>4,
    df['day_pct']>df_pct_amt5['p5max_abs_day_pct']*1.2,
    df['day_pct']>df_pct_amt5['p5mean_abs_day_pct']*1.5,
    df['day_pct']> 0.6*df['pct'],
#     (df['high']/df['close']-1)*100/df['day_pct']>0.2,
#     (df['high']/df['close']-1)*100/df['day_pct']<0.4,
    # 放量
    df['amt']>df_pct_amt5['p5max_abs_amt']*1.2,
    df['amt']>df_pct_amt5['p5mean_abs_amt']*1.5,
]


# 1.05 1.05
# mean      1.197287e-03
# median   -1.531735e-02
# max       3.320968e+00
# min      -8.727273e-01
# std       8.419110e-02
# size      1.392888e+06
# Name: r, dtype: float64
# mask_low_pos = [
#     df['close/60ma']<1.05,
#     df['close/30ma']<1.05,
# ]
mask_low_pos = [
    df['close/60ma']<1.05,
    df['close/30ma']<1.05,
    df[['close/30ma','close/60ma']].max(axis=1)>0.9,
]

# # mean          -0.003468
# # median        -0.021442
# # max            1.371940
# # min           -0.869820
# # std            0.081980
# # size      147259.000000
# # Name: r, dtype: float64
mask_basic =[   
    # 流动性、市值等基本限制 
#     mean     -2.309262e-03
#     median   -1.832455e-02
#     size   2373852
    df['amt']>=2,
    df['turnover_rate']<10,
    df['circ_mv']>50e4, 
    df['total_mv']>100e4,
    (df['pb']<10) | (df['pe']>0),
    df['close0']>10,
]

# -0.01,-0.02
# mean          -0.005841
# median        -0.022346
# max            2.528292
# min           -0.794109
# std            0.089689
# size      611120.000000
# Name: r, dtype: float64
# 
# 0.1,0.2
# mean          -0.005212
# median        -0.022130
# max            2.351421
# min           -0.794109
# std            0.090181
# size      333558.000000
# Name: r, dtype: float64
#
# 0.2 0.4
# mean          -0.005586
# median        -0.022727
# max            2.351421
# min           -0.794109
# std            0.091789
# size      201061.000000
# Name: r, dtype: float64
# mask_trend = [   
#     df['60ma_pct']>0.1,
#     df['30ma_pct']>0.2,
# ]
mask_trend = [   
    df['60ma_pct']>0,
    df['30ma_pct']>-0.2,        
]
print(time.time()-t0)
mask_final = True
# masks = mask_big_rise+mask_basic+mask_trend
masks = mask_low_pos+mask_trend+mask_big_rise+mask_basic

for m in masks+[mask_date2]:

    mask_final &= m
    m.sum(),len(m),round(m.sum()/len(m),4),round(mask_final.sum()/len(mask_final),4)

# df_baseline
# df_baseline_q
df_baseline2
df_baseline_q2


df.loc[mask_final,'r'].agg(agg_operations)
df.loc[mask_final,'r'].reset_index('code').resample('Q').agg(agg_operations)
tmp = df.loc[mask_final,r_columns].sort_values('date',ascending=False)
tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]

22700.504620552063


(2716624, 4593560, 0.5914, 0.5914)

(3384975, 4593560, 0.7369, 0.558)

(4048272, 4593560, 0.8813, 0.4918)

(1759403, 4593560, 0.383, 0.1529)

...

mean     -0.001853
median   -0.018078
max       3.320968
min      -0.967228
std       0.085897
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2018-03-31 -0.006932 -0.022059  2.229388 -0.967228  0.086282  190955
2018-06-30 -0.022454 -0.030781  1.119939 -0.950667  0.067774  197214
2018-09-30 -0.020828 -0.027933  1.655462 -0.869820  0.058617  218246
2018-12-31  0.003390 -0.016175  2.749064 -0.612903  0.083702  210707
2019-03-31  0.044422  0.014218  3.320968 -0.746753  0.117071  206870
2019-06-30 -0.011983 -0.022901  1.897051 -0.861940  0.067602  215321
2019-09-30  0.000302 -0.017704  2.243787 -0.876993  0.075026  236958
2019-12-31  0.010891 -0.003697  2.125000 -0.731959  0.078266  225939
2020-03-31 -0.012337 -0.023256  1.891414 -0.763889  0.110282  218582
2020-06-30 -0.005392 -0.020305  1.417069 -0.787500  0.072459  148256

mean        0.003390
median     -0.017610
max         0.313485
min        -0.655048
std         0.087915
size      251.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2018-03-31 -0.026122 -0.029172  0.087824 -0.089674  0.039636   26
2018-06-30 -0.045914 -0.035132  0.140234 -0.655048  0.148826   22
2018-09-30 -0.018151 -0.021405  0.056288 -0.099437  0.040355   12
2018-12-31 -0.018465 -0.021164  0.042090 -0.058170  0.037332    9
2019-03-31  0.033299 -0.001948  0.289901 -0.075323  0.110614    9
2019-06-30 -0.007570 -0.033629  0.148528 -0.121991  0.070994   28
2019-09-30  0.033568 -0.005293  0.313485 -0.060884  0.092561   39
2019-12-31  0.013372 -0.002145  0.155932 -0.090204  0.065469   38
2020-03-31 -0.003797 -0.025349  0.270503 -0.174593  0.088771   49
2020-06-30  0.066762  0.068657  0.240400 -0.039840  0.069947   19

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
601881.SH,2019-08-19,11.418000,11.698260,627.0,626,True,10.982039,2019-08-27,-0.038182
600999.SH,2019-08-19,26.245699,26.652418,2332.0,2326,True,25.295340,2019-09-02,-0.036210
600271.SH,2019-08-19,419.881744,428.442993,2190.0,2187,True,405.232483,2019-08-29,-0.034889
600958.SH,2019-08-19,11.148120,11.223300,1073.0,1072,True,10.707780,2019-09-02,-0.039499
000776.SZ,2019-08-19,71.454102,80.804749,2310.0,2293,True,77.615799,2019-09-18,0.086233
600837.SH,2019-08-19,309.962708,374.566376,2350.0,2331,True,351.156006,2019-09-18,0.132898
002812.SZ,2019-08-19,112.662392,119.184952,633.0,630,True,109.871994,2019-08-27,-0.024768
300188.SZ,2019-08-19,230.037506,261.585510,1968.0,1949,True,244.628464,2019-09-18,0.063429
000712.SZ,2019-08-19,162.809387,164.387497,2083.0,2082,True,156.628403,2019-08-27,-0.037965


In [68]:
df_r_tmp = df[['open','close']].copy()
df_r_tmp[['f1_open','f1_close']] = df[['open','close']].sort_index().groupby('code').shift(-1)
df_r_tmp[['f2_open','f2_close']] = df[['open','close']].sort_index().groupby('code').shift(-2)
df_r_tmp[['f3_open','f3_close']] = df[['open','close']].sort_index().groupby('code').shift(-3)

df_r_tmp.head()

open       close     f1_open    f1_close  \
code      date                                                         
000001.SZ 2014-01-02  707.650452  714.072998  709.402039  696.556946   
          2014-01-03  709.402039  696.556946  694.221436  681.376282   
          2014-01-06  694.221436  681.376282  673.202087  679.040833   
          2014-01-07  673.202087  679.040833  679.624695  686.631165   
          2014-01-08  679.624695  686.631165  682.544006  690.134338   

                         f2_open    f2_close     f3_open    f3_close  
code      date                                                        
000001.SZ 2014-01-02  694.221436  681.376282  673.202087  679.040833  
          2014-01-03  673.202087  679.040833  679.624695  686.631165  
          2014-01-06  679.624695  686.631165  682.544006  690.134338  
          2014-01-07  682.544006  690.134338  687.798828  690.134338  
          2014-01-08  687.798828  690.134338  688.966614  677.289246

In [89]:
df_r_tmp.head()
df[r_columns].head()
df_r_tmp['r2'] = (df_r_tmp['f3_open']/df_r_tmp['f1_open']-1)
df_r_tmp['r3'] = (df_r_tmp['f2_open']/df_r_tmp['f1_open']-1)

df_r_tmp['is_cut'] = (df_r_tmp['f2_close']<(df_r_tmp['f1_open']*0.98)).astype('uint8')
df_r_tmp['is_cut2'] = (df_r_tmp['f1_close']<(df_r_tmp['f1_open']*0.97)).astype('uint8')
df_r_tmp.head()



open       close     f1_open    f1_close  \
code      date                                                         
000001.SZ 2014-01-02  707.650452  714.072998  709.402039  696.556946   
          2014-01-03  709.402039  696.556946  694.221436  681.376282   
          2014-01-06  694.221436  681.376282  673.202087  679.040833   
          2014-01-07  673.202087  679.040833  679.624695  686.631165   
          2014-01-08  679.624695  686.631165  682.544006  690.134338   

                         f2_open    f2_close     f3_open    f3_close  \
code      date                                                         
000001.SZ 2014-01-02  694.221436  681.376282  673.202087  679.040833   
          2014-01-03  673.202087  679.040833  679.624695  686.631165   
          2014-01-06  679.624695  686.631165  682.544006  690.134338   
          2014-01-07  682.544006  690.134338  687.798828  690.134338   
          2014-01-08  687.798828  690.134338  688.966614  677.289246   

                            r2  is_cut  
code      date                          
000001.SZ 2014-01-02 -0.051029       1  
          2014-01-03 -0.021026       1  
          2014-01-06  0.013877       0  
          2014-01-07  0.012027       0  
          2014-01-08  0.009410       0

buy_at         max  max_idx  idx  is_selled  \
code      date                                                          
000001.SZ 2014-01-02  709.402039  709.985901    899.0  898       True   
          2014-01-03  694.221436  700.644043    900.0  899       True   
          2014-01-06  673.202087  700.060120    903.0  900       True   
          2014-01-07  679.624695  700.060120    903.0  901       True   
          2014-01-08  682.544006  700.060120    903.0  902       True   

                         sell_at  sell_date         r  
code      date                                         
000001.SZ 2014-01-02  680.792419 2014-01-16 -0.040329  
          2014-01-03  678.456970 2014-01-17 -0.022708  
          2014-01-06  660.940857 2014-01-21 -0.018213  
          2014-01-07  660.940857 2014-01-21 -0.027491  
          2014-01-08  660.940857 2014-01-21 -0.031651

open       close     f1_open    f1_close  \
code      date                                                         
000001.SZ 2014-01-02  707.650452  714.072998  709.402039  696.556946   
          2014-01-03  709.402039  696.556946  694.221436  681.376282   
          2014-01-06  694.221436  681.376282  673.202087  679.040833   
          2014-01-07  673.202087  679.040833  679.624695  686.631165   
          2014-01-08  679.624695  686.631165  682.544006  690.134338   

                         f2_open    f2_close     f3_open    f3_close  \
code      date                                                         
000001.SZ 2014-01-02  694.221436  681.376282  673.202087  679.040833   
          2014-01-03  673.202087  679.040833  679.624695  686.631165   
          2014-01-06  679.624695  686.631165  682.544006  690.134338   
          2014-01-07  682.544006  690.134338  687.798828  690.134338   
          2014-01-08  687.798828  690.134338  688.966614  677.289246   

                            r2  is_cut        r3  is_cut2  
code      date                                             
000001.SZ 2014-01-02 -0.051029       1 -0.021399        0  
          2014-01-03 -0.021026       1 -0.030278        0  
          2014-01-06  0.013877       0  0.009540        0  
          2014-01-07  0.012027       0  0.004295        0  
          2014-01-08  0.009410       0  0.007699        0

In [93]:
# df.loc[mask_final,'r'].agg(agg_operations)
# df.loc[mask_final,'r'].reset_index('code').resample('Q').agg(agg_operations)
# tmp = df.loc[mask_final,r_columns].sort_values('date',ascending=False)
# tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]




tmp2 = tmp.join(df_r_tmp,how='left')
tmp2.shape

mask_cut = tmp2['is_cut']==1
mask_cut2 = tmp2['is_cut2']==1

tmp2.loc[mask_cut,r_columns]
tmp2.loc[mask_cut2,r_columns]

tmp2.loc[mask_cut,['r','r2']].agg(agg_operations)
tmp2.loc[mask_cut,['r','r2']].reset_index('code').resample('Q').agg(agg_operations)


tmp2.loc[mask_cut2,['r','r3']].agg(agg_operations)
tmp2.loc[mask_cut2,['r','r3']].reset_index('code').resample('Q').agg(agg_operations)

(251, 20)

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
002389.SZ,2020-04-27,76.629753,87.401001,2167.0,2159,True,81.861504,2020-05-14,0.068273
600893.SH,2020-04-08,76.474808,80.449654,2344.0,2333,True,77.591705,2020-04-28,0.014605
300463.SZ,2020-03-26,91.679428,92.963364,1177.0,1176,True,86.451958,2020-03-31,-0.057019
002100.SZ,2020-03-26,141.599121,142.959564,2391.0,2390,True,133.209747,2020-03-31,-0.059247
300661.SZ,2020-03-24,445.055969,445.924011,674.0,673,True,418.692017,2020-03-27,-0.059237
300012.SZ,2020-03-10,316.671173,326.911316,2376.0,2375,True,293.099548,2020-03-13,-0.074436
300357.SZ,2020-03-05,260.407013,267.542969,1489.0,1488,True,249.108490,2020-03-10,-0.043388
600132.SH,2020-03-05,266.998566,274.605499,2428.0,2421,True,251.353043,2020-03-19,-0.058598
000002.SZ,2020-02-26,4486.494629,4853.072754,2315.0,2309,True,4541.407227,2020-03-10,0.012240


,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
002299.SZ,2020-05-11,55.292801,64.149124,2393.0,2388,True,60.668163,2020-05-21,0.097216
002389.SZ,2020-04-27,76.629753,87.401001,2167.0,2159,True,81.861504,2020-05-14,0.068273
300463.SZ,2020-03-26,91.679428,92.963364,1177.0,1176,True,86.451958,2020-03-31,-0.057019
002100.SZ,2020-03-26,141.599121,142.959564,2391.0,2390,True,133.209747,2020-03-31,-0.059247
300661.SZ,2020-03-24,445.055969,445.924011,674.0,673,True,418.692017,2020-03-27,-0.059237
603737.SH,2020-02-04,159.289993,164.555939,891.0,890,True,155.673172,2020-02-07,-0.022706
002153.SZ,2019-08-29,611.505127,733.494263,2235.0,2227,True,690.174255,2019-09-24,0.128648
300073.SZ,2019-08-07,98.150139,99.296181,2147.0,2146,True,92.174362,2019-08-12,-0.060884
600549.SH,2019-05-16,185.340118,196.372269,2259.0,2256,True,180.408096,2019-05-27,-0.026611


,r,r2
mean,-0.039459,-0.042039
median,-0.049923,-0.040332
max,0.078217,-0.010916
min,-0.135405,-0.121991
std,0.042412,0.021912
size,52.000000,52.000000


r                                                     r2  \
                mean    median       max       min       std size      mean   
date                                                                          
2018-03-31 -0.053799 -0.054023 -0.034230 -0.089674  0.022566    5 -0.046770   
2018-06-30 -0.063613 -0.055446 -0.034286 -0.135405  0.036362    6 -0.035012   
2018-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2018-12-31 -0.024402 -0.024402  0.009366 -0.058170  0.047755    2 -0.044715   
2019-03-31 -0.075323 -0.075323 -0.075323 -0.075323       NaN    1 -0.032281   
2019-06-30 -0.041270 -0.048857  0.078217 -0.121991  0.057840    9 -0.050372   
2019-09-30 -0.015703 -0.036041  0.063429 -0.060884  0.048631    8 -0.032097   
2019-12-31 -0.053674 -0.057895 -0.029746 -0.075671  0.017326    5 -0.040334   
2020-03-31 -0.042466 -0.049990  0.016480 -0.074436  0.027470   14 -0.045681   
2020-06-30  0.041439  0.041439  0.068273  0.014605  0.037949    2 -0.034545   

                                                         
              median       max       min       std size  
date                                                     
2018-03-31 -0.036169 -0.032184 -0.089674  0.024238    5  
2018-06-30 -0.031178 -0.016838 -0.058062  0.017990    6  
2018-09-30       NaN       NaN       NaN       NaN    0  
2018-12-31 -0.044715 -0.032680 -0.056749  0.017020    2  
2019-03-31 -0.032281 -0.032281 -0.032281       NaN    1  
2019-06-30 -0.038809 -0.016894 -0.121991  0.036351    9  
2019-09-30 -0.028172 -0.014405 -0.060884  0.017391    8  
2019-12-31 -0.044656 -0.018373 -0.060403  0.020074    5  
2020-03-31 -0.049301 -0.010916 -0.074436  0.016403   14  
2020-06-30 -0.034545 -0.024914 -0.044177  0.013621    2

,r,r3
mean,-0.025662,-0.050795
median,-0.053333,-0.049799
max,0.128648,-0.003268
min,-0.121991,-0.118343
std,0.070904,0.026824
size,21.000000,21.000000


r                                                     r3  \
                mean    median       max       min       std size      mean   
date                                                                          
2018-03-31 -0.089674 -0.089674 -0.089674 -0.089674       NaN    1 -0.067029   
2018-06-30 -0.055698 -0.055698 -0.053333 -0.058062  0.003344    2 -0.040988   
2018-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2018-12-31 -0.024402 -0.024402  0.009366 -0.058170  0.047755    2 -0.022571   
2019-03-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2019-06-30 -0.040509 -0.048646  0.078217 -0.121991  0.074030    8 -0.071588   
2019-09-30  0.033882  0.033882  0.128648 -0.060884  0.134020    2 -0.028606   
2019-12-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2020-03-31 -0.049552 -0.058128 -0.022706 -0.059247  0.017928    4 -0.043925   
2020-06-30  0.082745  0.082745  0.097216  0.068273  0.020466    2 -0.033465   

                                                         
              median       max       min       std size  
date                                                     
2018-03-31 -0.067029 -0.067029 -0.067029       NaN    1  
2018-06-30 -0.040988 -0.038839 -0.043137  0.003040    2  
2018-09-30       NaN       NaN       NaN       NaN    0  
2018-12-31 -0.022571 -0.003268 -0.041873  0.027298    2  
2019-03-31       NaN       NaN       NaN       NaN    0  
2019-06-30 -0.062683 -0.047046 -0.118343  0.025767    8  
2019-09-30 -0.028606 -0.026353 -0.030859  0.003186    2  
2019-12-31       NaN       NaN       NaN       NaN    0  
2020-03-31 -0.042354 -0.025342 -0.065652  0.019693    4  
2020-06-30 -0.033465 -0.017131 -0.049799  0.023100    2

In [99]:
# tmp2.loc[~mask_cut,r_columns]
tmp2.loc[~mask_cut & ~mask_cut2,['r','r2','r3']].agg(agg_operations)
tmp2.loc[~mask_cut & ~mask_cut2,['r','r2','r3']].reset_index('code').resample('Q').agg(agg_operations)

(tmp2.loc[~mask_cut & ~mask_cut2,'f2_open']/tmp2.loc[~mask_cut & ~mask_cut2,'f1_open']).agg(agg_operations)
(tmp2.loc[~mask_cut & ~mask_cut2,'f3_open']/tmp2.loc[~mask_cut & ~mask_cut2,'f2_open']).agg(agg_operations)

,r,r2,r3
mean,0.014068,0.018310,0.011491
median,-0.005357,0.009444,0.007460
max,0.313485,0.150000,0.131507
min,-0.655048,-0.031979,-0.051364
std,0.093171,0.032582,0.024119
size,195.000000,195.000000,195.000000


r                                                     r2  \
                mean    median       max       min       std size      mean   
date                                                                          
2018-03-31 -0.019532 -0.025070  0.087824 -0.070476  0.040309   21  0.007655   
2018-06-30 -0.039278 -0.018952  0.140234 -0.655048  0.174345   16  0.012254   
2018-09-30 -0.018151 -0.021405  0.056288 -0.099437  0.040355   12  0.016801   
2018-12-31 -0.016768 -0.021164  0.042090 -0.056940  0.038249    7  0.019458   
2019-03-31  0.046877  0.004538  0.289901 -0.048804  0.109941    8  0.005199   
2019-06-30  0.012664 -0.013917  0.148528 -0.054861  0.069793   17  0.019380   
2019-09-30  0.043538  0.000037  0.313485 -0.045455  0.097833   30  0.014314   
2019-12-31  0.023530  0.015441  0.155932 -0.090204  0.064106   33  0.024558   
2020-03-31  0.011671 -0.005422  0.270503 -0.174593  0.099873   35  0.019650   
2020-06-30  0.068108  0.069041  0.240400 -0.039840  0.075355   16  0.037260   

                                                               r3            \
              median       max       min       std size      mean    median   
date                                                                          
2018-03-31  0.005609  0.069521 -0.030797  0.025842   21  0.011365  0.007702   
2018-06-30  0.010117  0.063672 -0.016026  0.022227   16  0.007712  0.001870   
2018-09-30  0.019159  0.050310 -0.012758  0.023459   12  0.013450  0.008394   
2018-12-31  0.014015  0.059701  0.000827  0.020761    7  0.020962  0.024943   
2019-03-31 -0.002959  0.050285 -0.025325  0.024418    8  0.000524 -0.001585   
2019-06-30  0.005660  0.150000 -0.031979  0.044892   17  0.010960  0.006711   
2019-09-30  0.001098  0.136163 -0.020455  0.034576   30  0.001965 -0.000659   
2019-12-31  0.014493  0.130137 -0.017115  0.035586   33  0.015034  0.011321   
2020-03-31  0.016370  0.111833 -0.029307  0.031355   35  0.011545  0.013523   
2020-06-30  0.032086  0.125898 -0.005111  0.036666   16  0.027293  0.018773   

                                               
                 max       min       std size  
date                                           
2018-03-31  0.052393 -0.025721  0.021710   21  
2018-06-30  0.060547 -0.016635  0.022341   16  
2018-09-30  0.042857 -0.003973  0.015508   12  
2018-12-31  0.052537 -0.007854  0.020429    7  
2019-03-31  0.036807 -0.021104  0.018937    8  
2019-06-30  0.041440 -0.028846  0.020013   17  
2019-09-30  0.068431 -0.051364  0.021899   30  
2019-12-31  0.131507 -0.012718  0.026602   33  
2020-03-31  0.096128 -0.037267  0.024273   35  
2020-06-30  0.097012 -0.033644  0.033762   16

mean        1.011491
median      1.007460
max         1.131507
min         0.948636
std         0.024119
size      195.000000
dtype: float64

mean        1.006811
median      1.003514
max         1.116421
min         0.952322
std         0.024444
size      195.000000
dtype: float64

In [63]:
tmp.loc[idx['600845.SH',:],:]
tmp.loc[idx['603288.SH',:],:]
tmp.loc[idx['002475.SZ',:],:]

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
600845.SH,2020-04-14,256.244415,321.485779,2448.0,2428,True,317.845459,2020-05-18,0.2404


,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
603288.SH,2019-08-16,403.305573,450.592407,1360.0,1349,True,436.52359,2019-09-16,0.082364


KeyError: '002475.SZ'

In [49]:
tmp[tmp['r'].notnull()]

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
300382.SZ,2020-05-08,75.956345,77.300705,1519.0,1514,True,72.150955,2020-05-22,-0.050100
603633.SH,2020-03-19,21.106750,22.054401,814.0,812,True,21.382431,2020-03-24,0.013061
603822.SH,2020-03-17,27.438000,27.937799,859.0,857,True,26.122200,2020-03-20,-0.047955
300311.SZ,2019-11-14,64.359726,72.441422,1682.0,1678,True,64.874008,2019-11-21,0.007991
600359.SH,2019-07-17,10.891040,12.173269,2214.0,2213,True,11.112659,2019-07-22,0.020349
603609.SH,2019-07-02,19.443199,20.917120,1184.0,1180,True,19.584320,2019-07-11,0.007258


In [174]:
# mean     -0.001853
# median   -0.018078
# size     4593560


# mean         -0.003998
# median       -0.021001
# max           1.964467
# min          -0.675676
# std           0.089355
# size      40895.000000
# Name: r, dtype: float64
mask_big_rise = [
    # -0.003998 40895
    # 实体大阳线
    df['day_pct']>4,
#     df['day_pct']> 1.2*df['pct'],
    df['day_pct']>df_pct_amt5['p5max_abs_day_pct']*0.6,
    df['day_pct']>df_pct_amt5['p5mean_abs_day_pct']*1,
#     ((df['day_pct']>4) & (df['day_pct']> 1.2*df['pct'])) | (df['day_pct']>6),
#     (df['high']/df['close']-1)*100/df['day_pct']>0.2,
#     (df['high']/df['close']-1)*100/df['day_pct']<0.4,
    # 放量
    df['amt']>df_pct_amt5['p5max_abs_amt']*0.6,
    df['amt']>df_pct_amt5['p5mean_abs_amt']*1,
]


# 1.05 1.05
# mean      1.197287e-03
# median   -1.531735e-02
# max       3.320968e+00
# min      -8.727273e-01
# std       8.419110e-02
# size      1.392888e+06
# Name: r, dtype: float64
# mask_low_pos = [
#     df['close/60ma']<1.05,
#     df['close/30ma']<1.05,
# ]
mask_low_pos = [
    df['close/60ma']<1.05,
    df['close/30ma']<1.05,
    df['close/5ma']<1.15,
    df[['close/30ma','close/60ma','close/120ma','close/250ma']].max(axis=1)>0.9,
]

# # mean          -0.003468
# # median        -0.021442
# # max            1.371940
# # min           -0.869820
# # std            0.081980
# # size      147259.000000
# # Name: r, dtype: float64
mask_basic =[   
    # 流动性、市值等基本限制 
#     mean     -2.309262e-03
#     median   -1.832455e-02
#     size   2373852
    df['amt']>=2,
    df['turnover_rate']<10,
    df['circ_mv']>50e4, 
    df['total_mv']>100e4,
    (df['pb']<10) | (df['pe']>0),
    df['close0']>20,
]

# -0.01,-0.02
# mean          -0.005841
# median        -0.022346
# max            2.528292
# min           -0.794109
# std            0.089689
# size      611120.000000
# Name: r, dtype: float64
# 
# 0.1,0.2
# mean          -0.005212
# median        -0.022130
# max            2.351421
# min           -0.794109
# std            0.090181
# size      333558.000000
# Name: r, dtype: float64
#
# 0.2 0.4
# mean          -0.005586
# median        -0.022727
# max            2.351421
# min           -0.794109
# std            0.091789
# size      201061.000000
# Name: r, dtype: float64
# mask_trend = [   
#     df['60ma_pct']>0.1,
#     df['30ma_pct']>0.2,
# ]
mask_trend = [   
#     df[['60ma_pct','30ma_pct']].max(axis=1)>-0.5,
#     df['30ma_pct']>-0.2,        
]
print(time.time()-t0)
mask_final = True
# masks = mask_big_rise+mask_basic+mask_trend
masks = mask_low_pos+mask_trend+mask_big_rise+mask_basic

for m in masks+[mask_date2]:

    mask_final &= m
    m.sum(),len(m),round(m.sum()/len(m),4),round(mask_final.sum()/len(mask_final),4)

# df_baseline
# df_baseline_q
df_baseline2
df_baseline_q2


df.loc[mask_final,'r'].agg(agg_operations)
df.loc[mask_final,'r'].reset_index('code').resample('Q').agg(agg_operations)
tmp = df.loc[mask_final,r_columns].sort_values('date',ascending=False)
tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]

27911.857218027115


(2716624, 4593560, 0.5914, 0.5914)

(3384975, 4593560, 0.7369, 0.558)

(4511129, 4593560, 0.9821, 0.5577)

(4110067, 4593560, 0.8947, 0.505)

...

mean     -0.001853
median   -0.018078
max       3.320968
min      -0.967228
std       0.085897
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2018-03-31 -0.006932 -0.022059  2.229388 -0.967228  0.086282  190955
2018-06-30 -0.022454 -0.030781  1.119939 -0.950667  0.067774  197214
2018-09-30 -0.020828 -0.027933  1.655462 -0.869820  0.058617  218246
2018-12-31  0.003390 -0.016175  2.749064 -0.612903  0.083702  210707
2019-03-31  0.044422  0.014218  3.320968 -0.746753  0.117071  206870
2019-06-30 -0.011983 -0.022901  1.897051 -0.861940  0.067602  215321
2019-09-30  0.000302 -0.017704  2.243787 -0.876993  0.075026  236958
2019-12-31  0.010891 -0.003697  2.125000 -0.731959  0.078266  225939
2020-03-31 -0.012337 -0.023256  1.891414 -0.763889  0.110282  218582
2020-06-30 -0.005392 -0.020305  1.417069 -0.787500  0.072459  148256

mean         0.012261
median      -0.011347
max          0.659888
min         -0.867660
std          0.098671
size      1466.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2018-03-31  0.014862 -0.012214  0.397098 -0.158373  0.092005  153
2018-06-30 -0.014204 -0.026162  0.160080 -0.655048  0.079222  150
2018-09-30 -0.021437 -0.019561  0.158824 -0.867660  0.080861  203
2018-12-31 -0.025869 -0.032603  0.180067 -0.216818  0.058209  138
2019-03-31  0.077986  0.023392  0.428910 -0.091097  0.125038   65
2019-06-30 -0.001070 -0.017736  0.226106 -0.161134  0.062750  137
2019-09-30  0.041306 -0.006130  0.659888 -0.107934  0.125350   76
2019-12-31  0.016559  0.001269  0.327206 -0.087143  0.072303   79
2020-03-31  0.047190 -0.000435  0.558753 -0.174593  0.135204  269
2020-06-30  0.019367  0.007960  0.240400 -0.185714  0.068459  196

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
600053.SH,2019-05-07,56.870216,59.891037,1948.0,1947,True,56.279186,2019-05-10,-0.010393
002008.SZ,2019-05-07,474.381622,486.811920,2243.0,2242,True,452.311432,2019-05-14,-0.046524
600053.SH,2019-04-30,53.083248,59.891037,1948.0,1945,True,56.279186,2019-05-10,0.060206
603260.SH,2019-04-30,48.992203,50.068623,367.0,366,True,45.441021,2019-05-08,-0.072485
600729.SH,2019-04-29,104.768799,106.313087,2095.0,2094,True,97.986084,2019-05-07,-0.064740
002411.SZ,2019-04-26,83.819923,91.735321,1862.0,1860,True,82.653435,2019-05-06,-0.013917
002812.SZ,2019-04-17,118.370071,122.769989,547.0,546,True,114.540504,2019-04-23,-0.032352
600498.SH,2019-04-16,116.090004,121.447998,2207.0,2203,True,114.304001,2019-04-24,-0.015385
002912.SZ,2019-04-16,168.944992,170.505722,341.0,340,True,159.596710,2019-04-24,-0.055333


In [ ]:
# 排序性

df['close0_rank'] = df['close0'].groupby('date').rank(ascending=False)
df['close0_pct'] = df['close0'].groupby('date').rank(ascending=False,pct=True)

In [207]:
# mean     -0.001853
# median   -0.018078
# size     4593560


mask_big_rise = [
    # -0.003998 40895
    # 实体大阳线
    df['day_pct']>4,
#     df['day_pct']> 1.1*df['pct'],
    df['day_pct']>df_pct_amt5['p5max_abs_day_pct']*0.6,
    df['day_pct']>df_pct_amt5['p5mean_abs_day_pct']*1,
#     ((df['day_pct']>4) & (df['day_pct']> 1.2*df['pct'])) | (df['day_pct']>6),
#     (df['high']/df['close']-1)*100/df['day_pct']>0.2,
#     (df['high']/df['close']-1)*100/df['day_pct']<0.4,
    # 放量
    df['amt']>df_pct_amt5['p5max_abs_amt']*0.6,
    df['amt']>df_pct_amt5['p5mean_abs_amt']*1,
]



mask_low_pos = [
    df['close/60ma']<1.05,
    df['close/30ma']<1.05,
    df['close/5ma']<1.15,
    df[['close/30ma','close/60ma','close/120ma','close/250ma']].max(axis=1)>0.9,
]


mask_basic =[   
    # 流动性、市值等基本限制 
#     mean     -2.309262e-03
#     median   -1.832455e-02
#     size   2373852
    df['amt']>=2,
    df['turnover_rate']<10,
    df['circ_mv']>50e4, 
    df['total_mv']>100e4,
    (df['pb']<10) | (df['pe']>0),
    df['close0']>5,
#     df['close0_rank']<200,
    df['close0_rank']>10,
    df['close0_pct']<0.04,
    df.index.get_level_values('code').str[:3]!='688',
]


mask_trend = [   
#     df[['60ma_pct','30ma_pct']].max(axis=1)>-0.5,
#     df['30ma_pct']>-0.2,        
]
print(time.time()-t0)
mask_final = True
# masks = mask_big_rise+mask_basic+mask_trend
masks = mask_low_pos+mask_trend+mask_big_rise+mask_basic

for m in masks+[mask_date]:

    mask_final &= m
    m.sum(),len(m),round(m.sum()/len(m),4),round(mask_final.sum()/len(mask_final),4)

# df_baseline
# df_baseline_q
df_baseline2
df_baseline_q2


df.loc[mask_final,'r'].agg(agg_operations)
df.loc[mask_final,'r'].reset_index('code').resample('Q').agg(agg_operations)
tmp = df.loc[mask_final,r_columns].sort_values('date',ascending=False)
tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]

30058.583129167557


(2716624, 4593560, 0.5914, 0.5914)

(3384975, 4593560, 0.7369, 0.558)

(4511129, 4593560, 0.9821, 0.5577)

(4110067, 4593560, 0.8947, 0.505)

...

mean     -0.001853
median   -0.018078
max       3.320968
min      -0.967228
std       0.085897
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2018-03-31 -0.006932 -0.022059  2.229388 -0.967228  0.086282  190955
2018-06-30 -0.022454 -0.030781  1.119939 -0.950667  0.067774  197214
2018-09-30 -0.020828 -0.027933  1.655462 -0.869820  0.058617  218246
2018-12-31  0.003390 -0.016175  2.749064 -0.612903  0.083702  210707
2019-03-31  0.044422  0.014218  3.320968 -0.746753  0.117071  206870
2019-06-30 -0.011983 -0.022901  1.897051 -0.861940  0.067602  215321
2019-09-30  0.000302 -0.017704  2.243787 -0.876993  0.075026  236958
2019-12-31  0.010891 -0.003697  2.125000 -0.731959  0.078266  225939
2020-03-31 -0.012337 -0.023256  1.891414 -0.763889  0.110282  218582
2020-06-30 -0.005392 -0.020305  1.417069 -0.787500  0.072459  148256

mean        0.019579
median     -0.007904
max         0.659888
min        -0.202516
std         0.097676
size      462.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2016-03-31  0.036897  0.023119  0.277243 -0.090278  0.088840   48
2016-06-30  0.023229  0.020673  0.230769 -0.090428  0.077784   21
2016-09-30 -0.007255 -0.000165  0.026144 -0.055360  0.029975    6
2016-12-31 -0.042334 -0.038750 -0.037783 -0.050468  0.007061    3
2017-03-31  0.042346  0.066897  0.079963 -0.019821  0.054234    3
2017-06-30 -0.013977 -0.031321  0.030683 -0.041294  0.038997    3
2017-09-30  0.042857 -0.012949  0.165217 -0.023697  0.106103    3
2017-12-31  0.003943 -0.022409  0.130560 -0.044521  0.056565   11
2018-03-31  0.035661 -0.003311  0.340325 -0.121747  0.113548   26
2018-06-30 -0.019088 -0.023092  0.106960 -0.085724  0.053821   21
2018-09-30 -0.014401 -0.019604  0.098299 -0.202516  0.050910   57
2018-12-31 -0.030520 -0.035608  0.086250 -0.123750  0.049687   50
2019-03-31  0.078043  0.041939  0.316288 -0.091097  0.123019   24
2019-06-30  0.006961 -0.010271  0.226106 -0.161134  0.079762   43
2019-09-30  0.066463  0.013794  0.659888 -0.107934  0.156143   31
2019-12-31 -0.003761 -0.020765  0.201265 -0.069595  0.061656   20
2020-03-31  0.060994  0.016283  0.427659 -0.174593  0.146566   53
2020-06-30  0.030067  0.023973  0.118730 -0.072342  0.050251   39

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
000423.SZ,2018-10-22,407.731140,412.412079,2129.0,2128,True,380.328003,2018-10-25,-0.067209
300308.SZ,2018-10-22,139.891357,142.600632,1422.0,1421,True,129.847198,2018-10-25,-0.071800
002371.SZ,2018-10-22,206.608948,207.652191,2010.0,2009,True,192.240692,2018-10-26,-0.069543
600570.SH,2018-10-22,753.810608,760.546204,2071.0,2070,True,680.722534,2018-10-29,-0.096958
000963.SZ,2018-10-19,389.253784,418.999817,2090.0,2089,True,378.213989,2018-10-25,-0.028361
601601.SH,2018-10-19,43.983799,46.475800,2134.0,2133,True,41.479343,2018-10-30,-0.056940
601336.SH,2018-10-19,50.802803,54.333702,1635.0,1634,True,51.066303,2018-10-29,0.005187
600009.SH,2018-10-19,159.870834,172.934433,2134.0,2133,True,150.849991,2018-10-25,-0.056426
000596.SZ,2018-10-19,245.883957,255.433502,2127.0,2126,True,228.354202,2018-10-24,-0.071293


In [288]:
# mean     -0.001853
# median   -0.018078
# size     4593560


mask_big_rise = [
    # 不在大阴线之后
#     df['prev_day_pct'].abs()<df['day_pct'],
#     df['prev_pct']>-5,
    
    # 实体大阳线
    (df['day_pct']>5) |(df['pct']>5),
#     df['day_pct']>5, # 单独反而弱
#     df['pct']>5, # 单独反而弱
#     df['day_pct']> 1.1*df['pct'],
    
    # 限制得强(1,1.5)反而弱
    df['day_pct']>df_pct_amt5['p5max_abs_day_pct']*0.6, 
    df['day_pct']>df_pct_amt5['p5mean_abs_day_pct']*1,
#     df['day_pct']<df_pct_amt5['p5max_abs_day_pct']*1.2, 
#     df['day_pct']<df_pct_amt5['p5mean_abs_day_pct']*1.5,
    
#     (df['day_pct']< df_pct_amt5['p5max_abs_day_pct']*1.2) | (df['day_pct']<df_pct_amt5['p5mean_abs_day_pct']*1.5) |(df['amt']<df_pct_amt5['p5max_abs_amt']*1.2) | (df['amt']<df_pct_amt5['p5mean_abs_amt']*1.5),
#     (df['amt']<df_pct_amt5['p5max_abs_amt']*1.2) | (df['amt']<df_pct_amt5['p5mean_abs_amt']*1.5),
#     ((df['day_pct']>4) & (df['day_pct']> 1.2*df['pct'])) | (df['day_pct']>6),
#     (df['high']/df['close']-1)*100/df['day_pct']>0.2,
#     (df['high']/df['close']-1)*100/df['day_pct']<0.4,
    # 放量 # 限制得强(1,1.5)反而弱，但比涨幅超5日之流要强
    df['amt']>df_pct_amt5['p5max_abs_amt']*0.6,
    df['amt']>df_pct_amt5['p5mean_abs_amt']*1,
#     df['amt']<df_pct_amt5['p5max_abs_amt']*1.2,
#     df['amt']<df_pct_amt5['p5mean_abs_amt']*1.5,
    
    (df['amt']<df_pct_amt5['p5max_abs_amt']*1.2) | (df['amt']<df_pct_amt5['p5mean_abs_amt']*1.5),
]



mask_low_pos = [
    df['close/60ma']<1.05,
    df['close/30ma']<1.05,
    df['close/5ma']<1.15,
    df[['close/30ma','close/60ma','close/120ma','close/250ma']].max(axis=1)>0.9,
]


mask_basic =[   
    # 流动性、市值等基本限制 
#     mean     -2.309262e-03
#     median   -1.832455e-02
#     size   2373852
    df['amt']>=2,
    df['turnover_rate']<10,
    df['circ_mv']>50e4, 
    df['total_mv']>100e4,
    (df['pb']<10) | (df['pe']>0),
    df['close0']>5,
#     df['close0_rank']<200,
    df['close0_rank']>10,
    df['close0_pct']<0.04,
    df.index.get_level_values('code').str[:3]!='688',
]


mask_trend = [   
#     df[['60ma_pct','30ma_pct']].max(axis=1)>-0.5,
#     df['30ma_pct']>-0.2,        
]
print(time.time()-t0)
mask_final = True
# masks = mask_big_rise+mask_basic+mask_trend
masks = mask_low_pos+mask_trend+mask_big_rise+mask_basic

for m in masks+[mask_date]:

    mask_final &= m
    m.sum(),len(m),round(m.sum()/len(m),4),round(mask_final.sum()/len(mask_final),4)

# df_baseline
# df_baseline_q
df_baseline2
df_baseline_q2


df.loc[mask_final,'r'].agg(agg_operations)
df.loc[mask_final,'r'].reset_index('code').resample('Q').agg(agg_operations)
# df.loc[mask_final,'r'].reset_index('code').groupby('date').agg(agg_operations)

tmp = df.loc[mask_final,r_columns].sort_values('date',ascending=False)
tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]

54.83457279205322


(2716624, 4593560, 0.5914, 0.5914)

(3384975, 4593560, 0.7369, 0.558)

(4511129, 4593560, 0.9821, 0.5577)

(4110067, 4593560, 0.8947, 0.505)

...

mean     -0.001853
median   -0.018078
max       3.320968
min      -0.967228
std       0.085897
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2018-03-31 -0.006932 -0.022059  2.229388 -0.967228  0.086282  190955
2018-06-30 -0.022454 -0.030781  1.119939 -0.950667  0.067774  197214
2018-09-30 -0.020828 -0.027933  1.655462 -0.869820  0.058617  218246
2018-12-31  0.003390 -0.016175  2.749064 -0.612903  0.083702  210707
2019-03-31  0.044422  0.014218  3.320968 -0.746753  0.117071  206870
2019-06-30 -0.011983 -0.022901  1.897051 -0.861940  0.067602  215321
2019-09-30  0.000302 -0.017704  2.243787 -0.876993  0.075026  236958
2019-12-31  0.010891 -0.003697  2.125000 -0.731959  0.078266  225939
2020-03-31 -0.012337 -0.023256  1.891414 -0.763889  0.110282  218582
2020-06-30 -0.005392 -0.020305  1.417069 -0.787500  0.072459  148256

mean        0.023040
median     -0.013638
max         0.644828
min        -0.202516
std         0.119926
size      178.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2016-03-31  0.010452 -0.026856  0.409926 -0.108957  0.100492   32
2016-06-30  0.070933  0.048297  0.221096 -0.056596  0.140223    3
2016-09-30       NaN       NaN       NaN       NaN       NaN    0
2016-12-31 -0.044609 -0.044609 -0.038750 -0.050468  0.008286    2
2017-03-31 -0.019821 -0.019821 -0.019821 -0.019821       NaN    1
2017-06-30       NaN       NaN       NaN       NaN       NaN    0
2017-09-30       NaN       NaN       NaN       NaN       NaN    0
2017-12-31  0.026348  0.026348  0.056364 -0.003668  0.042449    2
2018-03-31  0.037073 -0.003091  0.304058 -0.056422  0.098171   12
2018-06-30 -0.023955 -0.054031  0.117836 -0.085724  0.064944    9
2018-09-30 -0.028491 -0.037435  0.084444 -0.202516  0.059446   23
2018-12-31 -0.034965 -0.041667  0.119010 -0.171442  0.055192   25
2019-03-31  0.056637  0.027185  0.334615 -0.057263  0.136054    8
2019-06-30  0.039005 -0.004572  0.280239 -0.080712  0.113965   11
2019-09-30  0.261278  0.226881  0.644828 -0.053478  0.322110    4
2019-12-31 -0.094992 -0.094992 -0.094992 -0.094992       NaN    1
2020-03-31  0.113133  0.114492  0.404529 -0.133762  0.167600   24
2020-06-30  0.037125  0.029275  0.189747 -0.057361  0.067281   21

,,sell_at,sell_at0,sell_date,sell_idx,buy_at,buy_at0,max,max_idx,idx,is_selled,r
code,date,,,,,,,,,,,
601888.SH,2018-08-27,135.181198,62.700001,2018-09-04,2043.0,137.983994,64.000000,143.503342,2040.0,2037,True,-0.020312
300601.SZ,2018-08-27,69.408539,46.180000,2018-09-03,388.0,73.030769,48.590000,74.834373,384.0,383,True,-0.049599
600009.SH,2018-08-21,163.521393,54.200001,2018-08-30,2103.0,174.978470,58.580002,175.619568,2098.0,2096,True,-0.065477
000963.SZ,2018-08-21,448.439148,43.869999,2018-09-07,2065.0,433.310577,42.389999,490.144928,2057.0,2052,True,0.034914
000596.SZ,2018-08-21,247.085999,74.000000,2018-09-06,2101.0,262.144897,78.510002,277.136993,2094.0,2089,True,-0.057445
600276.SH,2018-08-21,2400.604492,65.139999,2018-09-03,2101.0,2452.567383,66.550003,2634.989502,2096.0,2092,True,-0.021187
601888.SH,2018-08-21,127.850792,59.299999,2018-09-06,2045.0,132.270584,61.349998,143.503342,2040.0,2033,True,-0.033415
300601.SZ,2018-08-13,64.117981,42.660000,2018-08-20,378.0,68.672066,45.689999,72.880470,376.0,373,True,-0.066316
600570.SH,2018-08-13,705.085205,49.200001,2018-08-20,2031.0,738.333130,51.520000,742.775757,2027.0,2026,True,-0.045031


In [294]:
start = 20160101
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
t0 = time.time()
df_r = pd.read_parquet(r"database\return_spc_5%_10%_60_8_inf%").loc[idx[:,start_date:],:].sort_index()
# df_r = pd.read_parquet(r"database\return_spc_5%_30%_60_8_inf%").loc[idx[:,start_date:],:].sort_index()
print(time.time()-t0)
df_r.shape
df_r.head()



C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


14.034038782119751


(3453794, 11)

sell_at  sell_at0  sell_date  sell_idx      buy_at  \
code      date                                                                
000001.SZ 2016-01-04  963.992798     11.21 2016-01-08    1390.0  969.152466   
          2016-01-05  963.992798     11.21 2016-01-08    1390.0  982.051514   
          2016-01-06  931.315063     10.83 2016-01-12    1392.0  981.191589   
          2016-01-07  910.676514     10.59 2016-01-14    1394.0  963.992798   
          2016-01-08  889.177979     10.34 2016-01-18    1396.0  945.934021   

                      buy_at0         max  max_idx   idx  is_selled         r  
code      date                                                                 
000001.SZ 2016-01-04    11.27  994.950562   1387.0  1386       True -0.005324  
          2016-01-05    11.42  994.090698   1388.0  1387       True -0.018389  
          2016-01-06    11.41  981.191589   1389.0  1388       True -0.050833  
          2016-01-07    11.21  970.872314   1390.0  1389       True -0.055308  
          2016-01-08    11.00  952.813538   1391.0  1390       True -0.060000

In [295]:
# tmp = tmp.drop(columns=r_columns).join(df_r,how='inner')

# tmp['r'].agg(agg_operations)
# tmp['r'].reset_index('code').resample('Q').agg(agg_operations)
# tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]

tmp.to_

mean        0.039801
median      0.003875
max         0.604815
min        -0.202516
std         0.124117
size      178.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2016-03-31  0.034131  0.017475  0.386581 -0.090278  0.102887   32
2016-06-30  0.044952  0.048297  0.143154 -0.056596  0.099917    3
2016-09-30       NaN       NaN       NaN       NaN       NaN    0
2016-12-31 -0.044609 -0.044609 -0.038750 -0.050468  0.008286    2
2017-03-31 -0.019821 -0.019821 -0.019821 -0.019821       NaN    1
2017-06-30       NaN       NaN       NaN       NaN       NaN    0
2017-09-30       NaN       NaN       NaN       NaN       NaN    0
2017-12-31  0.022773  0.022773  0.056551 -0.011005  0.047769    2
2018-03-31  0.081994  0.053081  0.340325 -0.056422  0.123329   12
2018-06-30 -0.016925 -0.011208  0.106960 -0.085724  0.062203    9
2018-09-30 -0.010660 -0.013832  0.078947 -0.202516  0.061329   23
2018-12-31 -0.015928 -0.019284  0.131122 -0.096168  0.051553   25
2019-03-31  0.038850  0.027185  0.217033 -0.057263  0.090124    8
2019-06-30  0.023976 -0.004572  0.138551 -0.080712  0.083178   11
2019-09-30  0.259404  0.233674  0.604815 -0.034548  0.287530    4
2019-12-31 -0.094992 -0.094992 -0.094992 -0.094992       NaN    1
2020-03-31  0.149191  0.117814  0.508252 -0.133762  0.195339   24
2020-06-30  0.034721  0.021462  0.189747 -0.031937  0.059769   21

sell_at    sell_at0  sell_date  sell_idx  \
code      date                                                       
300601.SZ 2018-08-13    64.117981   42.660000 2018-08-20     378.0   
000568.SZ 2018-11-02  1276.047363   40.150002 2018-11-30    2128.0   
          2018-10-12  1287.171021   40.500000 2018-10-19    2098.0   
603589.SH 2018-07-31    55.579498   53.700001 2018-08-03     758.0   
002049.SZ 2018-07-12   199.587997   45.330002 2018-07-20    1809.0   
601888.SH 2018-09-07   137.337189   63.700001 2018-10-09    2062.0   
300347.SZ 2020-04-07   948.979431   76.709999 2020-05-12    1739.0   
600536.SH 2019-05-08   264.549988   50.000000 2019-05-15    2254.0   
603259.SH 2020-02-03   155.807220  110.580002 2020-02-27     440.0   
603288.SH 2020-03-20   495.143585  126.699997 2020-04-13    1506.0   
603068.SH 2020-05-06    77.053802   76.900002 2020-05-19     265.0   
300359.SZ 2016-02-16   214.125381   64.709999 2016-02-26     406.0   
002714.SZ 2020-02-04   950.142822  125.199997 2020-03-12    1465.0   
000596.SZ 2018-08-27   261.744202   78.389999 2018-09-03    2098.0   
300226.SZ 2020-02-06   377.799347   96.010002 2020-02-20    2001.0   
300558.SZ 2017-12-13    62.196030   62.009998 2017-12-19     275.0   
300033.SZ 2016-02-02   560.688477   68.309998 2016-02-19    1466.0   
002594.SZ 2018-01-22    63.634560   62.880001 2018-01-30    1592.0   
002095.SZ 2016-01-18   305.301758   69.639999 2016-01-22    1467.0   
300725.SZ 2020-04-09   162.513992   83.000000 2020-04-14     584.0   
603156.SH 2019-02-13    66.021324   45.880001 2019-03-04     252.0   
603160.SH 2020-04-28   239.991653  235.979996 2020-05-15     872.0   
300367.SZ 2016-02-01   342.570190   68.349998 2016-02-22     450.0   
600332.SH 2018-10-19    48.557705   33.009998 2018-10-25    1951.0   
002304.SZ 2018-03-29   410.895172  104.660004 2018-04-04    1995.0   
600809.SH 2017-12-11   243.083344   56.610001 2017-12-27    1935.0   
000963.SZ 2018-08-09   439.034912   42.950001 2018-08-17    2050.0   
002230.SZ 2018-03-26   508.696960   57.080002 2018-04-09    1922.0   
300725.SZ 2020-03-25   148.807999   76.000000 2020-03-31     575.0   
601888.SH 2019-02-11   132.593994   61.500000 2019-03-07    2162.0   
002371.SZ 2018-10-19   198.452698   41.849998 2018-10-24    2011.0   
300347.SZ 2018-10-19   345.769348   42.090000 2018-10-25    1365.0   
603019.SH 2018-10-19    84.752640   42.040001 2018-10-26     955.0   
603659.SH 2020-04-14    69.306900   68.349998 2020-04-22     544.0   
300347.SZ 2019-06-20   640.694092   51.790001 2019-07-09    1536.0   
002739.SZ 2016-01-08   194.679001   97.000000 2016-01-14     193.0   
603019.SH 2018-06-29    93.360962   46.310001 2018-07-06     881.0   
300364.SZ 2016-03-17   101.900002  101.900002 2016-03-23     272.0   
300601.SZ 2018-08-27    69.769257   46.419998 2018-08-31     387.0   
300033.SZ 2016-03-14   627.803894   75.430000 2016-03-25    1491.0   
603259.SH 2020-04-09   140.265961   99.550003 2020-04-17     475.0   
000938.SZ 2018-06-26   149.171005   42.020000 2018-07-06    1861.0   
000418.SZ 2018-04-25   188.961121   66.959999 2018-05-18    2001.0   
000596.SZ 2018-10-19   228.354202   68.389999 2018-10-24    2129.0   
603019.SH 2016-02-16    66.376312   66.309998 2016-02-24     302.0   
600570.SH 2019-01-04   848.395203   59.200001 2019-01-17    2131.0   
300450.SZ 2018-04-27   419.853668   69.339996 2018-05-15     581.0   
603658.SH 2020-03-25   131.796005  126.000000 2020-04-20     881.0   
600009.SH 2018-10-30   156.582306   51.900002 2018-11-08    2147.0   
300122.SZ 2018-10-19   160.397507   38.650002 2018-10-24    1885.0   

                           buy_at     buy_at0          max  max_idx   idx  \
code      date                                                              
300601.SZ 2018-08-13    68.672066   45.689999    72.880470    376.0   373   
000568.SZ 2018-11-02  1271.280029   40.000000  1327.851929   2119.0  2108   
          2018-10-12  1387.284302   43.650002  1400.950562   2094.0  2093   


In [276]:
mask2 = df['prev_day_pct']<-8
tmp2 = df.loc[mask_final & mask2,r_columns].sort_values('date',ascending=False)
tmp2['r'].agg(agg_operations)
tmp2['r'].reset_index('code').resample('Q').agg(agg_operations)

mean      0.039027
median   -0.019553
max       0.376816
min      -0.133762
std       0.179938
size      6.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2017-03-31 -0.019821 -0.019821 -0.019821 -0.019821       NaN    1
2017-06-30       NaN       NaN       NaN       NaN       NaN    0
2017-09-30       NaN       NaN       NaN       NaN       NaN    0
2017-12-31       NaN       NaN       NaN       NaN       NaN    0
2018-03-31       NaN       NaN       NaN       NaN       NaN    0
2018-06-30       NaN       NaN       NaN       NaN       NaN    0
2018-09-30       NaN       NaN       NaN       NaN       NaN    0
2018-12-31  0.033131  0.033131  0.085547 -0.019284  0.074127    2
2019-03-31       NaN       NaN       NaN       NaN       NaN    0
2019-06-30 -0.055333 -0.055333 -0.055333 -0.055333       NaN    1
2019-09-30  0.376816  0.376816  0.376816  0.376816       NaN    1
2019-12-31       NaN       NaN       NaN       NaN       NaN    0
2020-03-31 -0.133762 -0.133762 -0.133762 -0.133762       NaN    1

In [277]:
tmp2 = tmp.join(df_r_tmp,how='left')
tmp2.shape

mask_cut = tmp2['is_cut']==1
mask_cut2 = tmp2['is_cut2']==1

tmp2.loc[mask_cut,r_columns]
tmp2.loc[mask_cut2,r_columns]

tmp2.loc[mask_cut,['r','r2']].agg(agg_operations)
tmp2.loc[mask_cut,['r','r2']].reset_index('code').resample('Q').agg(agg_operations)


tmp2.loc[mask_cut2,['r','r3']].agg(agg_operations)
tmp2.loc[mask_cut2,['r','r3']].reset_index('code').resample('Q').agg(agg_operations)

# tmp2.loc[~mask_cut,r_columns]
tmp2.loc[~mask_cut & ~mask_cut2,['r','r2','r3']].agg(agg_operations)
tmp2.loc[~mask_cut & ~mask_cut2,['r','r2','r3']].reset_index('code').resample('Q').agg(agg_operations)

# (tmp2.loc[~mask_cut & ~mask_cut2,'f2_open']/tmp2.loc[~mask_cut & ~mask_cut2,'f1_open']).agg(agg_operations)
# (tmp2.loc[~mask_cut & ~mask_cut2,'f3_open']/tmp2.loc[~mask_cut & ~mask_cut2,'f2_open']).agg(agg_operations)

(178, 20)

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
603883.SH,2020-05-18,83.278366,87.455727,1150.0,1141,False,NaN,NaT,NaN
300725.SZ,2020-04-09,157.638580,161.260880,582.0,581,True,162.513992,2020-04-14,0.030928
002594.SZ,2020-03-31,61.098000,61.873199,2117.0,2116,True,57.721798,2020-04-03,-0.055259
603501.SH,2020-03-30,160.629959,173.691986,542.0,537,True,163.722275,2020-04-10,0.019251
600570.SH,2020-03-13,1716.751831,1716.751831,2410.0,2409,True,1651.298340,2020-03-18,-0.038126
300709.SZ,2020-03-10,118.100395,119.112396,582.0,581,True,102.303070,2020-03-13,-0.133762
603883.SH,2019-11-28,75.978325,76.381584,1031.0,1030,True,68.761002,2019-12-03,-0.094992
600132.SH,2019-07-19,227.992691,234.466690,2277.0,2270,True,220.115997,2019-08-01,-0.034548
600600.SH,2019-06-11,80.331360,80.513626,2286.0,2285,True,75.294075,2019-06-18,-0.062706


,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
300454.SZ,2020-04-14,166.592514,198.786774,478.0,466,True,186.371994,2020-05-15,0.118730
002594.SZ,2020-03-31,61.098000,61.873199,2117.0,2116,True,57.721798,2020-04-03,-0.055259
603737.SH,2020-03-20,153.480606,200.480515,943.0,923,True,197.688248,2020-04-21,0.288034
600570.SH,2020-03-13,1716.751831,1716.751831,2410.0,2409,True,1651.298340,2020-03-18,-0.038126
300709.SZ,2020-03-10,118.100395,119.112396,582.0,581,True,102.303070,2020-03-13,-0.133762
603883.SH,2019-11-28,75.978325,76.381584,1031.0,1030,True,68.761002,2019-12-03,-0.094992
601888.SH,2018-10-31,119.140549,125.522316,2080.0,2078,True,118.903397,2018-11-14,-0.001991
600585.SH,2018-10-12,155.441406,155.792664,2121.0,2120,True,145.781189,2018-10-19,-0.062147
000568.SZ,2018-10-12,1387.284302,1400.950562,2094.0,2093,True,1287.171021,2018-10-19,-0.072165


,r,r2
mean,-0.039617,-0.049803
median,-0.054405,-0.038126
max,0.386581,0.030928
min,-0.202516,-0.202516
std,0.089921,0.040363
size,37.000000,37.000000


r                                                     r2  \
                mean    median       max       min       std size      mean   
date                                                                          
2016-03-31  0.035722 -0.035411  0.386581 -0.090278  0.173344    7 -0.049825   
2016-06-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2016-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2016-12-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-03-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-06-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-12-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2018-03-31 -0.026690 -0.026690 -0.026690 -0.026690       NaN    1 -0.026318   
2018-06-30 -0.076734 -0.080419 -0.064060 -0.085724  0.011293    3 -0.067364   
2018-09-30 -0.079108 -0.058177 -0.044659 -0.202516  0.061204    6 -0.069002   
2018-12-31 -0.051127 -0.053551  0.003750 -0.096168  0.030218    9 -0.043754   
2019-03-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2019-06-30 -0.059361 -0.060042 -0.055333 -0.062706  0.003733    3 -0.023231   
2019-09-30 -0.034548 -0.034548 -0.034548 -0.034548       NaN    1 -0.023190   
2019-12-31 -0.094992 -0.094992 -0.094992 -0.094992       NaN    1 -0.094992   
2020-03-31 -0.051974 -0.046693  0.019251 -0.133762  0.063153    4 -0.068334   
2020-06-30  0.030928  0.030928  0.030928  0.030928       NaN    2  0.001744   

                                                         
              median       max       min       std size  
date                                                     
2016-03-31 -0.046428 -0.011029 -0.090278  0.030045    7  
2016-06-30       NaN       NaN       NaN       NaN    0  
2016-09-30       NaN       NaN       NaN       NaN    0  
2016-12-31       NaN       NaN       NaN       NaN    0  
2017-03-31       NaN       NaN       NaN       NaN    0  
2017-06-30       NaN       NaN       NaN       NaN    0  
2017-09-30       NaN       NaN       NaN       NaN    0  
2017-12-31       NaN       NaN       NaN       NaN    0  
2018-03-31 -0.026318 -0.026318 -0.026318       NaN    1  
2018-06-30 -0.080419 -0.035949 -0.085724  0.027335    3  
2018-09-30 -0.046595 -0.022433 -0.202516  0.067204    6  
2018-12-31 -0.033898 -0.015120 -0.096168  0.027006    9  
2019-03-31       NaN       NaN       NaN       NaN    0  
2019-06-30 -0.019389 -0.015922 -0.034381  0.009811    3  
2019-09-30 -0.023190 -0.023190 -0.023190       NaN    1  
2019-12-31 -0.094992 -0.094992 -0.094992       NaN    1  
2020-03-31 -0.050725 -0.038126 -0.133762  0.044176    4  
2020-06-30  0.001744  0.030928 -0.027440  0.041272    2

,r,r3
mean,-0.041523,-0.056725
median,-0.064117,-0.054246
max,0.288034,-0.011762
min,-0.202516,-0.124590
std,0.099270,0.031366
size,20.000000,20.000000


r                                                     r3  \
                mean    median       max       min       std size      mean   
date                                                                          
2016-03-31 -0.048861 -0.059326  0.022800 -0.090278  0.040137    6 -0.067970   
2016-06-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2016-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2016-12-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-03-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-06-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-12-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2018-03-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2018-06-30 -0.083072 -0.083072 -0.080419 -0.085724  0.003752    2 -0.053438   
2018-09-30 -0.106493 -0.066087 -0.050877 -0.202516  0.083505    3 -0.081507   
2018-12-31 -0.045434 -0.062147 -0.001991 -0.072165  0.037955    3 -0.024721   
2019-03-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2019-06-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2019-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2019-12-31 -0.094992 -0.094992 -0.094992 -0.094992       NaN    1 -0.074578   
2020-03-31  0.015222 -0.046693  0.288034 -0.133762  0.186580    4 -0.049193   
2020-06-30  0.118730  0.118730  0.118730  0.118730       NaN    1 -0.029773   

                                                         
              median       max       min       std size  
date                                                     
2016-03-31 -0.068357 -0.020643 -0.124590  0.035332    6  
2016-06-30       NaN       NaN       NaN       NaN    0  
2016-09-30       NaN       NaN       NaN       NaN    0  
2016-12-31       NaN       NaN       NaN       NaN    0  
2017-03-31       NaN       NaN       NaN       NaN    0  
2017-06-30       NaN       NaN       NaN       NaN    0  
2017-09-30       NaN       NaN       NaN       NaN    0  
2017-12-31       NaN       NaN       NaN       NaN    0  
2018-03-31       NaN       NaN       NaN       NaN    0  
2018-06-30 -0.053438 -0.051805 -0.055070  0.002309    2  
2018-09-30 -0.103482 -0.030087 -0.110951  0.044687    3  
2018-12-31 -0.027119 -0.011762 -0.035281  0.011941    3  
2019-03-31       NaN       NaN       NaN       NaN    0  
2019-06-30       NaN       NaN       NaN       NaN    0  
2019-09-30       NaN       NaN       NaN       NaN    0  
2019-12-31 -0.074578 -0.074578 -0.074578       NaN    1  
2020-03-31 -0.055578 -0.020147 -0.065467  0.019995    4  
2020-06-30 -0.029773 -0.029773 -0.029773       NaN    1

,r,r2,r3
mean,0.059077,0.036070,0.024524
median,0.021835,0.028972,0.023430
max,0.659888,0.160990,0.162632
min,-0.080712,-0.034669,-0.046159
std,0.117487,0.042968,0.034337
size,136.000000,136.000000,136.000000


r                                                     r2  \
                mean    median       max       min       std size      mean   
date                                                                          
2016-03-31  0.038609  0.048544  0.199123 -0.068653  0.078826   23  0.044216   
2016-06-30  0.044952  0.048297  0.143154 -0.056596  0.099917    3  0.075579   
2016-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2016-12-31 -0.044609 -0.044609 -0.038750 -0.050468  0.008286    2  0.006226   
2017-03-31 -0.019821 -0.019821 -0.019821 -0.019821       NaN    1  0.026607   
2017-06-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-09-30       NaN       NaN       NaN       NaN       NaN    0       NaN   
2017-12-31  0.022773  0.022773  0.056551 -0.011005  0.047769    2  0.023100   
2018-03-31  0.092711  0.061667  0.340325 -0.056422  0.123823   11  0.041183   
2018-06-30  0.012979  0.003493  0.106960 -0.055175  0.054041    6  0.013283   
2018-09-30  0.015339  0.008403  0.082961 -0.066316  0.042790   17  0.022182   
2018-12-31  0.004262  0.001539  0.131122 -0.056426  0.052794   15  0.028264   
2019-03-31  0.038850  0.027185  0.217033 -0.057263  0.090124    8  0.032687   
2019-06-30  0.048514  0.064256  0.138551 -0.080712  0.074101    8  0.033089   
2019-09-30  0.375745  0.376816  0.659888  0.090532  0.284680    3  0.076067   
2019-12-31       NaN       NaN       NaN       NaN       NaN    0       NaN   
2020-03-31  0.164633  0.201434  0.427659 -0.065126  0.160195   19  0.039950   
2020-06-30  0.037619  0.011997  0.189747 -0.031937  0.064793   18  0.040777   

                                                               r3            \
              median       max       min       std size      mean    median   
date                                                                          
2016-03-31  0.042654  0.135117 -0.027285  0.050322   23  0.028661  0.028155   
2016-06-30  0.037588  0.152516  0.036633  0.066631    3  0.004117 -0.001286   
2016-09-30       NaN       NaN       NaN       NaN    0       NaN       NaN   
2016-12-31  0.006226  0.022727 -0.010276  0.023337    2  0.006178  0.006178   
2017-03-31  0.026607  0.026607  0.026607       NaN    1  0.030536  0.030536   
2017-06-30       NaN       NaN       NaN       NaN    0       NaN       NaN   
2017-09-30       NaN       NaN       NaN       NaN    0       NaN       NaN   
2017-12-31  0.023100  0.035834  0.010367  0.018008    2  0.011834  0.011834   
2018-03-31  0.028972  0.152715 -0.001801  0.044126   11  0.017013  0.010284   
2018-06-30  0.015779  0.052522 -0.028355  0.033213    6 -0.000001 -0.005706   
2018-09-30  0.024590  0.079678 -0.022005  0.033182   17  0.011961  0.006969   
2018-12-31  0.022834  0.135334 -0.034669  0.044135   15  0.039759  0.043970   
2019-03-31  0.024865  0.079670  0.005866  0.025355    8  0.016441  0.016433   
2019-06-30  0.014622  0.115595 -0.004088  0.044246    8  0.021935  0.026416   
2019-09-30  0.112252  0.132241 -0.016293  0.080608    3  0.060170  0.079971   
2019-12-31       NaN       NaN       NaN       NaN    0       NaN       NaN   
2020-03-31  0.032563  0.160990 -0.025292  0.045222   19  0.034777  0.031746   
2020-06-30  0.035838  0.118509 -0.022363  0.037428   18  0.025738  0.020940   

                                               
                 max       min       std size  
date                                           
2016-03-31  0.130634 -0.046159  0.050513   23  
2016-06-30  0.038710 -0.025071  0.032232    3  
2016-09-30       NaN       NaN       NaN    0  
2016-12-31  0.010795  0.001561  0.006530    2  
2017-03-31  0.030536  0.030536       NaN    1  
2017-06-30       NaN       NaN       NaN    0  
2017-09-30       NaN       NaN       NaN    0  
2017-12-31  0.028708 -0.005039  0.023863    2  
2018-03-31  0.062341 -0.008673  0.020898   11  
2018-06-30  0.041826 -0.018528  0.021252    6  
2018-09-30  0.051170 -0.023146  0.025631   17  
2018-12-31  0.111111 -0.001

In [292]:
# tmp2.shape
tmp.to_csv('strategy_detail2.csv')
# tmp.loc[idx['603737.SH',:]]

In [198]:
tmp_day = df.loc[idx[:,'2020-05-25'],['close','close0']]
# df['close0'].groupby('date').rank().tail()
tmp_day.head(10)
tmp_day['rank1'] = tmp_day['close0'].rank(ascending=False)
tmp_day['pct1'] = tmp_day['close0'].rank(ascending=False,pct=True)

tmp_day['rank2'] = tmp_day['close0'].groupby('date').rank(ascending=False)
tmp_day.head(20)

,,close,close0
code,date,,
000001.SZ,2020-05-25,1414.830200,12.960000
000002.SZ,2020-05-25,3778.569336,25.459999
000004.SZ,2020-05-25,112.572807,27.700001
000005.SZ,2020-05-25,23.262678,2.510000
000006.SZ,2020-05-25,167.210709,4.720000
000007.SZ,2020-05-25,66.272003,8.000000
000008.SZ,2020-05-25,65.882355,2.950000
000009.SZ,2020-05-25,57.760681,6.520000
000010.SZ,2020-05-25,42.668999,3.960000


,,close,close0,rank1,pct1,rank2
code,date,,,,,
000001.SZ,2020-05-25,1414.830200,12.960000,1384.5,0.362719,1384.5
000002.SZ,2020-05-25,3778.569336,25.459999,628.0,0.164527,628.0
000004.SZ,2020-05-25,112.572807,27.700001,555.5,0.145533,555.5
000005.SZ,2020-05-25,23.262678,2.510000,3621.0,0.948651,3621.0
000006.SZ,2020-05-25,167.210709,4.720000,2961.0,0.775740,2961.0
000007.SZ,2020-05-25,66.272003,8.000000,2096.5,0.549253,2096.5
000008.SZ,2020-05-25,65.882355,2.950000,3515.0,0.920880,3515.0
000009.SZ,2020-05-25,57.760681,6.520000,2425.5,0.635447,2425.5
000010.SZ,2020-05-25,42.668999,3.960000,3211.0,0.841237,3211.0


In [121]:
tmp_code = tmp.groupby('code')['r'].agg(agg_operations)
tmp_code['median*size'] = tmp_code['median'] * tmp_code['size']
tmp_code['mean*size'] = tmp_code['mean'] * tmp_code['size']
tmp_code['accum_r'] = tmp[['median*size','mean*size']]
tmp_code[tmp_code['size']>10].sort_values('median*size',ascending=False).head(50)

,mean,median,max,min,std,size,median*size
code,,,,,,,
002049.SZ,0.075981,0.065911,0.348438,-0.056281,0.121125,11,0.725020
000997.SZ,0.018724,0.034338,0.122692,-0.129771,0.069085,14,0.480734
002202.SZ,0.043496,0.043200,0.117567,-0.047929,0.050152,11,0.475200
000063.SZ,0.042406,0.025231,0.294273,-0.071795,0.100785,17,0.428922
000858.SZ,0.033292,0.023333,0.149918,-0.051783,0.062685,16,0.373326
600872.SH,0.019702,0.019560,0.101923,-0.048490,0.043391,17,0.332517
600383.SH,0.052589,0.023829,0.343724,-0.064063,0.111185,13,0.309778
000002.SZ,0.040598,0.020408,0.188314,-0.037487,0.070810,15,0.306122
002508.SZ,0.019406,0.015773,0.141343,-0.075363,0.064172,15,0.236593


In [257]:
# df.loc[('600460.SH','2020-04-02'),['60ma_pct','30ma_pct']]
df['prev_pct'] = df['pct'].sort_index().groupby('code').shift(1)
df['prev_day_pct'] = df['day_pct'].sort_index().groupby('code').shift(1)

In [83]:
df.columns
df['close/30ma'] = df['close']/df['30ma']

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '10ma', '20ma', '40ma', '60ma', '120ma', '250ma',
       'close/5ma', 'close/10ma', 'close/20ma', 'close/40ma', 'close/60ma',
       'close/120ma', 'close/250ma', 'buy_at', 'max', 'max_idx', 'idx',
       'is_selled', 'sell_at', 'sell_date', 'r', '60ma_pct', '20ma_pct',
       'p5max_abs_pct', 'p10max_abs_pct', '30ma', '30ma_pct'],
      dtype='object')

In [290]:
df.loc[mask_date2,'r'].agg(agg_operations[:-1])


mean     -0.003412
median   -0.023810
max       2.763278
min      -0.967228
std       0.100699
Name: r, dtype: float64

NameError: name 'feature_engineering' is not defined